# Import Packages

In [1]:
import pandas as pd
import numpy as np
import os
from re import sub
import re
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
df = pd.read_csv("D:DataSet/csvdata/zomato.csv")

Remove unnecessary Columns

In [3]:
A = df.drop(labels=["url","address","name","phone","location","menu_item","listed_in(city)","dish_liked"],axis=1)

In [4]:
A.isna().sum()

online_order                      0
book_table                        0
rate                           7775
votes                             0
rest_type                       227
cuisines                         45
approx_cost(for two people)     346
reviews_list                      0
listed_in(type)                   0
dtype: int64

# EDA

In [5]:
A.rate = A.rate.apply(lambda x: str(x).split("/")[0])

In [6]:
A.dropna(inplace=True)

In [7]:
A.rate = A.rate.apply(lambda x: sub("[^0-9. ]","0",x)).astype("float")

In [9]:
k = A.reviews_list.apply(lambda x:sub("[()]","",x))

In [10]:
rev_list = []

In [11]:
for i in k:
    c = i.split("/")
    s = sub("[^0-5.]","",i)
    try:        
        m = str(s).replace(" ","")[0]
        rev_list.append(m)
    except:
        rev_list.append(0)


In [12]:
A["reviews"] = rev_list

In [13]:
A.drop(labels=["reviews_list"],axis=1,inplace=True)

In [14]:
A.sample(5)

,online_order,book_table,rate,votes,rest_type,cuisines,approx_cost(for two people),listed_in(type),reviews
29310,Yes,No,4.4,949,Cafe,"Cafe, European, Continental",900,Cafes,4
19661,Yes,Yes,4.2,543,"Casual Dining, Pub","North Indian, Chinese, Pizza","1,000",Delivery,2
18411,No,No,3.1,63,Sweet Shop,"Mithai, Street Food",150,Delivery,3
24918,Yes,No,3.9,24,Quick Bites,"South Indian, North Indian, Chinese",300,Dine-out,1
33967,Yes,No,4.4,2873,Cafe,"Cafe, American, Burger, Sandwich, Beverages",600,Dine-out,3


In [15]:
A["reviews"] = A["reviews"].astype("int")

In [16]:
A["approx_cost(for two people)"] = A["approx_cost(for two people)"].apply(lambda x: sub("[^0-9]","",x)).astype("int")

In [17]:
A.sample(5)

,online_order,book_table,rate,votes,rest_type,cuisines,approx_cost(for two people),listed_in(type),reviews
18928,No,No,3.6,12,Quick Bites,"South Indian, North Indian",300,Dine-out,2
14488,No,No,3.3,4,Quick Bites,"North Indian, Chinese",200,Dine-out,5
48540,No,No,3.0,22,Quick Bites,"Street Food, Rolls",200,Dine-out,1
47723,No,No,3.5,14,Quick Bites,"Desserts, Fast Food",250,Delivery,1
18195,Yes,Yes,4.1,2075,Casual Dining,Rajasthani,800,Delivery,4


# Preprocessing

In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder,StandardScaler
le = LabelEncoder()

In [19]:
A["online_order"] = le.fit_transform(A.online_order)

In [20]:
A["book_table"] = le.fit_transform(A.book_table)

In [21]:
Y = A.rate

In [22]:
X = A.drop(labels=["rate"],axis=1)

In [23]:
X.head(1)

,online_order,book_table,votes,rest_type,cuisines,approx_cost(for two people),listed_in(type),reviews
0,1,1,775,Casual Dining,"North Indian, Mughlai, Chinese",800,Buffet,4


In [49]:
ss = StandardScaler()

# Split Data

In [50]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.2,random_state=23)

In [51]:
ohe = OneHotEncoder(sparse=False,handle_unknown="ignore")

In [52]:
xtrain = ohe.fit_transform(xtrain)
xtest = ohe.transform(xtest)

# Traning Model

In [112]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor

In [116]:
rig = Ridge(alpha=0.001)

In [117]:
model = rig.fit(xtrain,ytrain)

In [126]:
rfc = RandomForestRegressor(n_estimators=100)

In [127]:
model_rf = rfc.fit(xtrain,ytrain)

In [2]:
from sklearn.metrics import r2_score,mean_absolute_error
def model_pred(model,xtrain,xtest,ytrain,ytest):
    m1_tr = model.predict(xtrain)
    m1_ts = model.predict(xtest)

    r2_tr = r2_score(ytrain,m1_tr)
    r2_ts = r2_score(ytest,m1_ts)

    err_tr = mean_absolute_error(ytrain,m1_tr)
    err_ts = mean_absolute_error(ytest,m1_ts)

    print(f"R2 score of Traning Data is {r2_tr} \nR2 score of Testing Data is {r2_ts}")
    print("------------------------Mean Absolute Error------------------------------")
    print(f"Error of Traning Data is {err_tr} \nError of Testing Data is {err_ts}")


In [120]:
model_pred(model_rf,xtrain,xtest,ytrain,ytest)

R2 score of Traning Data is 0.9962830001519148 
R2 score of Testing Data is 0.982105191342106
------------------------Mean Absolute Error------------------------------
Error of Traning Data is 0.0258217367525162 
Error of Testing Data is 0.06462039771621898


In [125]:
model_pred(model_rf,xtrain,xtest,ytrain,ytest)

R2 score of Traning Data is 0.9964474216894094 
R2 score of Testing Data is 0.983119956374085
------------------------Mean Absolute Error------------------------------
Error of Traning Data is 0.025135416955205582 
Error of Testing Data is 0.06280861690552342


In [128]:
model_pred(model_rf,xtrain,xtest,ytrain,ytest)

R2 score of Traning Data is 0.9965628014262243 
R2 score of Testing Data is 0.9830636660482418
------------------------Mean Absolute Error------------------------------
Error of Traning Data is 0.024899432028996894 
Error of Testing Data is 0.0627313661772686


# Export Model Using Pickle

In [121]:
import pickle

In [129]:
pickle.dump(X,open("data.pkl","wb"))
pickle.dump(model_rf,open("model.pkl","wb"))
pickle.dump(ohe,open("ohe.pkl","wb"))